In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

### Training Data

In [2]:
datadir = 'C:/Users/LightSpeed/Light Speed/Deep Learning/Brain Tumor Classification MRI/Training'
categories = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
train_images = []
train_labels = []
for category in categories:
    path = os.path.join(datadir,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        img_array = cv2.resize(img_array,(100,100) )
        train_images.append(img_array)
        train_labels.append(categories.index(category))

In [3]:
len(train_images),len(train_labels)

(2870, 2870)

In [4]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
train_images.shape[1:], train_labels.shape

((100, 100, 3), (2870,))

In [5]:
train_images = train_images/255.0

In [6]:
train_labels

array([0, 0, 0, ..., 3, 3, 3])

### Testing Data

In [7]:
datadir = os.path.normpath('C:/Users/LightSpeed/Light Speed/Deep Learning/Brain Tumor Classification MRI/Testing')
categories = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
test_images = []
test_labels = []
for category in categories:
    path = os.path.join(datadir,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        img_array = cv2.resize(img_array,(100,100) )
        test_images.append(img_array)
        test_labels.append(categories.index(category))

In [8]:
len(test_images),len(test_labels)

(394, 394)

In [9]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_images.shape[1:], test_labels.shape

((100, 100, 3), (394,))

In [10]:
test_images = test_images/255.0

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.python.keras.layers import Conv2D,MaxPooling2D
from tensorflow.python.keras import regularizers

from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers

In [12]:
weight_decay = 1e-4
model = Sequential()

#1st Convolutional Layer
model.add(Conv2D(32,(3,3),padding='same',kernel_regularizer=regularizers.l2(weight_decay) ,input_shape=train_images.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#2nd Convolutional Layer
model.add(Conv2D(64,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#3rd Convolutional Layer
model.add(Conv2D(128,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#4th Convolutional Layer
model.add(Conv2D(256,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#5th Convolutional Layer
model.add(Conv2D(512,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#6th Convolutional Layer
model.add(Conv2D(1024,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#Fully connected layer
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(4,activation='softmax'))

In [13]:
sgd = keras.optimizers.SGD(0.1)

model.compile(optimizer=sgd,
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [14]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [15]:
model.fit(train_images,train_labels,
         epochs=100,
         validation_data=(test_images,test_labels))

Epoch 1/100
90/90 [==============================] - 10s 107ms/step - loss: 1.7170 - accuracy: 0.4275 - val_loss: 1.5127 - val_accuracy: 0.3452
Epoch 2/100
90/90 [==============================] - 5s 55ms/step - loss: 1.3084 - accuracy: 0.4700 - val_loss: 2.9380 - val_accuracy: 0.2766
Epoch 3/100
90/90 [==============================] - 5s 55ms/step - loss: 1.3111 - accuracy: 0.4610 - val_loss: 3.9125 - val_accuracy: 0.2690
Epoch 4/100
90/90 [==============================] - 5s 58ms/step - loss: 1.2611 - accuracy: 0.5056 - val_loss: 3.1666 - val_accuracy: 0.3046
Epoch 5/100
90/90 [==============================] - 5s 58ms/step - loss: 1.1966 - accuracy: 0.5328 - val_loss: 2.9480 - val_accuracy: 0.2843
Epoch 6/100
90/90 [==============================] - 5s 57ms/step - loss: 1.1233 - accuracy: 0.5645 - val_loss: 2.3156 - val_accuracy: 0.3959
Epoch 7/100
90/90 [==============================] - 5s 56ms/step - loss: 1.1188 - accuracy: 0.5815 - val_loss: 2.6919 - val_accuracy: 0.2690
Epoc

90/90 [==============================] - 5s 57ms/step - loss: 0.4151 - accuracy: 0.9446 - val_loss: 1.6481 - val_accuracy: 0.7081
Epoch 59/100
90/90 [==============================] - 5s 57ms/step - loss: 0.4161 - accuracy: 0.9418 - val_loss: 2.5884 - val_accuracy: 0.7157
Epoch 60/100
90/90 [==============================] - 5s 57ms/step - loss: 0.4442 - accuracy: 0.9380 - val_loss: 2.0080 - val_accuracy: 0.7259
Epoch 61/100
90/90 [==============================] - 5s 56ms/step - loss: 0.4193 - accuracy: 0.9456 - val_loss: 2.1105 - val_accuracy: 0.7487
Epoch 62/100
90/90 [==============================] - 5s 57ms/step - loss: 0.4257 - accuracy: 0.9463 - val_loss: 1.8634 - val_accuracy: 0.7183
Epoch 63/100
90/90 [==============================] - 5s 59ms/step - loss: 0.4116 - accuracy: 0.9443 - val_loss: 1.8227 - val_accuracy: 0.7234
Epoch 64/100
90/90 [==============================] - 5s 57ms/step - loss: 0.4008 - accuracy: 0.9516 - val_loss: 3.1485 - val_accuracy: 0.6954
Epoch 65/100

In [16]:
model.evaluate(test_images,test_labels)

13/13 [==============================] - 0s 16ms/step - loss: 2.3829 - accuracy: 0.7310


[2.382873773574829, 0.7309644818305969]

In [20]:
yprob_train = model.predict(train_images)
ypred_train = [np.argmax(i) for i in yprob_train]

yprob_test = model.predict(test_images)
ypred_test = [np.argmax(i) for i in yprob_test]

In [22]:
from sklearn.metrics import confusion_matrix

conf_mx_train = confusion_matrix(train_labels, ypred_train)
print(conf_mx_train)
print('*************************************************')
conf_mx_test = confusion_matrix(test_labels, ypred_test)
print(conf_mx_test)

[[801  12   9   4]
 [  3 818   1   0]
 [  0   0 395   0]
 [  0   1   0 826]]
*************************************************
[[ 18  55  24   3]
 [  0 114   1   0]
 [  0   0 105   0]
 [  3  12   8  51]]
